## 2층 신경망 구성하여 오차 역전파를 수동으로 구현해보기
- 은닉층 포함된 MLP에서 오차 역전파 흐름 전체를 자동 미분 없이 수동미분으로 구현한다.
- forward(순전파) -> loss(손실 계산) - > backward(역전파) - > gradient(기울기)업데이트

In [1]:
import torch

x = torch.tensor([[0.5, 0.8]])  # shape : (1, 2)
y_true = torch.tensor([[1.0]])  # shape : (1, 1)

print(f"x.size = {x.size()}, y.true_size = {y_true.size()}")

x.size = torch.Size([1, 2]), y.true_size = torch.Size([1, 1])


### 1. 입력층 가중치/편향

In [2]:
# 가중치/편향 초기화
W1 = torch.tensor([[0.1, 0.3],
                    [0.2, 0.4]], requires_grad= False)  # 입력(2) -> 은닉노드 (2) 각각 가중치 행렬
b1 = torch.tensor([0.1, 0.2], requires_grad= False) # 은닉층 편향 백터

print(W1)
print(b1)

tensor([[0.1000, 0.3000],
        [0.2000, 0.4000]])
tensor([0.1000, 0.2000])


- requires_grad= False : Pytorch가 .backward()(역전파)시 자동으로 gradient를 계산하지 않는다는 의미

### 2. 은닉층 가중치/편향

In [3]:
# 가중치/편향 초기화
W2 = torch.tensor([[0.5],
                    [0.6]], requires_grad= False)  # 은닉(2) -> 출력(1) 가중치 행렬, shape : (2,1)
b2 = torch.tensor([0.3], requires_grad= False) # 출력층 편향 스칼라, shape : (1,1)

# 출력노드가 1개이므로 편향도 1개임.

print(W2)
print(b2)

tensor([[0.5000],
        [0.6000]])
tensor([0.3000])


### 3. 활성화 함수 정의

In [4]:
# 시그모이드 활성화 함수 : 입력을 0~1로 압축해서 반환 (비선형성 부여)
def sigmoid(x):
    return 1 / (1 + torch.exp(-x))


# 시그모이드 미분 함수  : 역전파에서 기울기 계산에 사용
def sigmoid_deriv(x):
    s = sigmoid(x)      # 순전파 시그모아드 출력값 s를 미분 계산
    return s * (1 - s)  # sigmoid'(x) = s * (1 - s)  => 기울기 계산


In [5]:
z1 = x @ W1 + b1    # 첫번째 은닉층 선형 결합 : (1,2) @ (2,2) + (1,2) = shape : (1,2)

a1 = sigmoid(z1)    # 첫번째 은닉층 활성화 함수 : (1,2)

z2 = a1 @ W2 + b2   # 출력층 선형 결합 : (1,2) @ (2,1) + (1,1) = shape : (1,1)

a2 = sigmoid(z2)    # 출력층 활성화함수 : 0~1 사이값으로 반환. shape : (1,1)

print(z1)
print(a1)
print(z2)
print(a2)


tensor([[0.3100, 0.6700]])
tensor([[0.5769, 0.6615]])
tensor([[0.9853]])
tensor([[0.7282]])


@ 연산자를 사용   
텐서끼리 연산할 시 행렬곱 수행    
(1,2) @  (2,2) => (1,2)   

### 4. 역전파

In [6]:
# 역전파

# MSE 평균제곱오차로 손실 계산 (0.5를 곱한건 미분시 2가 상쇄되어 깔끔하게 약분된다. 기울기 계산 단순화)
loss = 0.5 * (y_true - a2) ** 2

#### 1) 출력층 오차항

In [7]:
# 기울기 계산
delta2 = (a2 - y_true) * sigmoid_deriv(z2)  # 출력층 오차항 = (손실의 미분)*(시그모이드 미분) -> shape : (1,1)

dW2 = a1.T @ delta2     # W2 기울기 : (a1 전치) @ (z2에 대한 기울기) -> shape : (2,1)

db2 = delta2    # b2 기울기 : 출력층 편향을 그대로 delta2가 된다. -> (1,1)

print(delta2)
print(dW2)
print(db2)

tensor([[-0.0538]])
tensor([[-0.0310],
        [-0.0356]])
tensor([[-0.0538]])


1. $ a2 - y\_true ={\frac {\partial L} {\partial a_2}} $ => 제곱 오차에서 예측 - 정답 방향의 오차 (손실의 미분)

    - sigmoid_deriv(z2) = ${\frac {\partial L} {\partial z_2}}$ : 출력층 활성화 함수의 기울기 (시그모이드 미분)

    이 둘을 곱해서 나온 delta2 = z2에 대한 기울기가 된다.  수식은 $\frac {\partial L} {\partial z_2} $

    

2. a1.shape : (1,2) @ delta2(1, 1) => $ a_1 $.T.shape : (2, 1) @ (1, 1) => (2, 1)

3. b2에 대한 기울기 : 은닉층의 노드가 여러개인 경우에는 평균을 구해서 사용하지만, 여기서는 샘플이 1개라서 delta2가 그대로 편향의 기울기 역활도 한다.

#### 2) 은닉층의 오차항

In [8]:
delta1 = delta2 @ W2.T * sigmoid_deriv(z1)  # 은닉층의 오차항(delta) = 출력층 오차가중치전파((출력층의 오차항(delta) * 가중치(w2)) * 시그모이드 미분)

dW1 = x.T @ delta1  # W1의 기울기 = 입력층의 입력값과 은닉층의 오차항의 내적

db1 = delta1    # b1의 기울기 = 은닉층의 오차항

print(delta1)
print(dW1)
print(db1)

tensor([[-0.0066, -0.0072]])
tensor([[-0.0033, -0.0036],
        [-0.0053, -0.0058]])
tensor([[-0.0066, -0.0072]])


1. delta2 @ W2.T : 출력층 오차(delta2)를 W2로 체인룰에 따라 은닉층으로 전달. 수식은 $ \frac {\partial L}{\partial a_1} $

    sigmoid_deriv(z2) : 은닉층 활성화함수 기울기 $\frac {\partial \sigma(a_1)}{\partial z_1} $
    둘을 곱해서 은닉층 선형 출력 z1에 대한 기울기 delta1 = $ \frac {\partial L}{\partial z_1} $

2. dw1 = a0.T @ delta1 : W1의 기울기 ! (1,2) @ (1,2) => 전치 (2,1) # (1,2) = shape : (2,2) 
|


In [9]:
print("출력층 기울기 dW2", dW2) # 출력층 가중치 (W2) 에 대한 기울기 계산
print("은닉층 기울기 dW1", dW1) # 은닉층 가중치 (W1) 에 대한 기울기 계산
print("은닉층 편향 기울기 db1", db1) # 은닉층 편향 (b1) 에 대한 기울기 계산

출력층 기울기 dW2 tensor([[-0.0310],
        [-0.0356]])
은닉층 기울기 dW1 tensor([[-0.0033, -0.0036],
        [-0.0053, -0.0058]])
은닉층 편향 기울기 db1 tensor([[-0.0066, -0.0072]])


출력층 가중치 (W2)에 대한 기울기 = $\frac{\partial L}{\partial W_2}$ 

은닉층 가중치 (W1)에 대한 기울기 = $\frac{\partial L}{\partial W_1}$ 

은닉층 편향 (b1)에 대한 기울기 = $\frac{\partial L}{\partial b_1}$ 
